In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 14 14:13:34 2022

@author: dmc lab

ION Grad Bootcamp 2022
Widefield Data Analysis
"""

import os
import numpy as np
import easygui
import tifffile
import glob
from skimage import io
from skimage.measure import block_reduce
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
from scipy.stats import pearsonr

In [ ]:
def import_multi_tif(tif_folder, n_channels=2, down_sample_factor= 2,dtype=np.uint8):
    tif_filepaths = glob.glob(os.path.join(tif_folder + '\*.tif'))
    print('\n' + str(len(tif_filepaths)) + ' file(s) to process')
    frame = 0 
    for i, image_path in enumerate(tif_filepaths):
        im = io.imread(image_path) #cv2 might also work for this
        im = np.array(im,dtype=dtype)
        file_frames = int(im.shape[0]/down_sample_factor)
        
        if n_channels==2:
            if i == 0:
                wf_blue = np.ndarray([len(tif_filepaths)*file_frames, int(im.shape[1]/down_sample_factor), int(im.shape[2]/down_sample_factor)])
                wf_green = np.ndarray([len(tif_filepaths)*file_frames, int(im.shape[1]/down_sample_factor), int(im.shape[2]/down_sample_factor)])
                   
            wf_green[frame:frame+file_frames]= np.atleast_3d(block_reduce(im[np.arange(1,len(im),2)],(1,down_sample_factor,down_sample_factor),func=np.mean))
            if file_frames == im.shape[0]/down_sample_factor:
                wf_blue[frame:frame+file_frames] = np.atleast_3d(block_reduce(im[np.arange(0,len(im),2)],(1,down_sample_factor,down_sample_factor),func=np.mean))
            else:
                wf_blue[frame:frame+file_frames+1] = np.atleast_3d(block_reduce(im[np.arange(0,len(im),2)],(1,down_sample_factor,down_sample_factor),func=np.mean))
        frame = frame+file_frames
        print('\nFile ' + str(i+1) + ' of ' + str(len(tif_filepaths)) + ' complete')
    return wf_blue, wf_green

In [ ]:
# PREPROCESS WIDEFIELD RECORDINGS

tif_folder = easygui.diropenbox() #change this if using hal or iq
wf_blue, wf_green = import_multi_tif(tif_folder, n_channels=2, down_sample_factor= 2,dtype=np.float16)

# add in trimming step HERE

# CALCULATE DF/F 
uF_blue = wf_blue.reshape(wf_blue.shape[0], wf_blue.shape[1]*wf_blue.shape[2]) #unfold images so df/f calculation is easier and faster
uF_green = wf_green.reshape(wf_green.shape[0], wf_green.shape[1]*wf_green.shape[2])

base_blue = np.mean(uF_blue, axis = 0) #calculate base as mean across entire video
base_green = np.median(uF_green, axis = 0)

uF_dFF_blue = np.divide(uF_blue - base_blue, base_blue) # calculate df/f
uF_dFF_green = np.divide(uF_green - base_green, base_green)

dFF_blue = uF_dFF_blue.reshape(wf_blue.shape) # reshape to size of original image
dFF_green = uF_dFF_green.reshape(wf_green.shape)


# SAVE VIDEO
os.chdir(tif_folder)
tifffile.imwrite('dFF_blue.tif', dFF_blue.astype('float32'))
tifffile.imwrite('dFF_green.tif', dFF_green.astype('float32'))
 

# Open in ImageJ and view your recording

In [ ]:
# PREPROCESS AROUSAL MEASURES
# BE SURE TO RUN TDMS TO TXT LABVIEW PROGRAM FIRST!!!!!!

lv_folder = easygui.diropenbox()
os.chdir(lv_folder)
files_list = [] #pull out names of files and directories so you can go into LabView folder
for root, directories, files in os.walk(lv_folder):
   for name in files:
      files_list.append(os.path.join(root, name))

os.chdir(root[:-9])  # change directory to LabView folder
frame_times = np.loadtxt('Frame_Clock_1.txt')
mouseID = np.loadtxt('Animal_ID.txt', dtype = str)
face_times = np.loadtxt('LV_FrameClock.txt')
walk_times = np.loadtxt('Encoder_Time.txt')
walk = np.loadtxt('Encoder_Value.txt')
whisk = np.loadtxt('Whisker_Energy.txt')
pupil = np.loadtxt('Pupil_Area.txt')
aud_times = np.loadtxt('Aud_Stim_Time.txt') # THESE FILES WILL ONLY EXIST IF YOU RAN THAT MODALITY
vis_times = np.loadtxt('Vis_Stim_Time.txt')

avi_start = np.where(face_times>frame_times[0])[0][0]
avi_end = np.where(face_times>frame_times[-1])[0][0]

walk_start = np.where(walk_times>frame_times[0])[0][0]
walk_end = np.where(walk_times>frame_times[-1])[0][0]

walk = walk[walk_start:walk_end]
pupil = pupil[avi_start:avi_end]
whisk = whisk[avi_start:avi_end]

plt.plot(pupil)
plt.savefig('Spont_pupil')

plt.plot(whisk)
plt.savefig('Spont_whisk')

plt.plot(walk)
plt.savefig('Spont_walk')

In [ ]:
# PREPROCESS FACE VIDEO

avi_folder = easygui.diropenbox()
os.chdir(avi_folder)

face_vid = io.imread('face_ds.tif')
face_vid = np.array(face_vid,dtype=np.float16)

face_vid = face_vid[avi_start:avi_end,:,:,0]
face_vid_save = face_vid.astype(dtype = 'uint16')
tifffile.imwrite('face_trim.tif',face_vid_save)

# Now combine your face video, widefield recording, and arousal measures into a slide

In [ ]:
# TRYING POINT CORRELATION


aud_frames=[] # find frame numbers for all auditory stims
for time in aud_times:
    temp = np.where(frame_times>time)[0]
    aud_frames = np.append(aud_frames,temp[0])

#create 4D variable that is # of stims, # of frames to clip, x, y
dFF_aud_clip = np.zeros((len(aud_frames), 60, int(dFF_blue.shape[1]),int(dFF_blue.shape[2]))) 
for i in range(len(aud_frames)):
    temp = dFF_blue[int(aud_frames[i])+30:int(aud_frames[i])+90,:,:] # clip to 1 s ater to 3 s after
    dFF_aud_clip[i,:,:,:] = temp
    
#average over first few stims to get auditory response to use for seeding
dFF_aud_avg = np.mean(np.mean(dFF_aud_clip[0:4,:,:,:], axis = 0),axis = 0)


fig, ax = subplots()
avg = plt.imshow(dFF_aud_avg, vmin = np.min(dFF_aud_avg), vmax = 0, cmap = 'viridis')
fig.colorbar(avg)
fig.show()




seedx = 185
seedy = 45

corr=np.full(avg_dFF.shape, np.nan)

for allx in range(int(avg_dFF.shape[0])):
    for ally in range(int(avg_dFF.shape[1])):
        if all(dFF_blue[:,allx, ally] == dFF_blue[:,allx, ally]):
            corr[allx,ally] = pearsonr(dFF_blue[:,seedx,seedy], dFF_blue[:,allx, ally])[0]
            
plt.imshow(corr)

os.chdir(tif_folder)
tifffile.imwrite('dFF_aud_avg.tif', dFF_aud_avg.astype('float32'))
tifffile.imwrite('corr_audSeed.tif', corr.astype('float32'))

#ImageJ
# image transform
# edit invert
#auto
# image threshold - manually adjust, apply, create mask, create outlines, select blood vessel image, shift-e, flatten, repeat with other modalities, save, add _MMM to name...